## <span style='font-family:Georgia'> Objectives
The purpose of this notebook is text mining on test data, aimed at creating a baseline rule-model, which is to be a benchmark of the neural model developed in the later phase of the project.

In [1]:
# loading packages
import sys
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath("__file__"))))
from supportive_functions import (
    find_nonalpha,
    flatten,
    count_punctuation,
    separate_special_chars,
    rm_consecutive_spaces
    )

import warnings
warnings.filterwarnings(action='ignore')
pd.options.display.max_columns = None

%matplotlib inline
plt.rcParams['figure.figsize'] = (9, 6)
sns.set(style="darkgrid")

## <span style='font-family:Georgia'> Data loading & preparation

In [2]:
# reading data
data_test_in = pd.read_csv('../../data/source/test-A/in.tsv', sep='\t', header=None)
data_test_exp = pd.read_csv('../../data/source/test-A/expected.tsv', sep='\t', header=None)

In [3]:
# renaming columns

data_test_in.columns = ['FileId', 'ASROutput']
data_test_exp.columns = ['FixedOutput']

# adding index to expected output (assumption: the same ordering of the records)

data_test_exp['FileId'] = data_test_in['FileId']
data_test_exp = data_test_exp[['FileId', 'FixedOutput']]

In [4]:
# data_test_exp.to_csv("../../data/outputs/eda/test/test_expected_with_ids.csv", index=False)
data_test_exp.shape

(200, 2)

## <span style='font-family:Georgia'> Separate special characters from words

In [5]:
data_test_in = separate_special_chars(data_test_in)
data_test_exp = separate_special_chars(data_test_exp)

In [6]:
data_test_exp["FixedOutput"] = data_test_exp["FixedOutput"].apply(rm_consecutive_spaces)
data_test_in["ASROutput"] = data_test_in["ASROutput"].apply(rm_consecutive_spaces)

## <span style='font-family:Georgia'> Check for records with non-character symbols excluded from target classes set

In [7]:
target_classes=['.', ',', '?', '!', '-', ':', '...']
numbers = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '0']

In [8]:
non_characters_test = np.unique(
    data_test_exp["FixedOutput"].apply(lambda x: find_nonalpha(x))
)
symbols_to_drop_test = list(
    set(flatten(non_characters_test)) - set(target_classes) - set(numbers)
)
print(f"List of symbols to be dropped: {str(symbols_to_drop_test)}")

List of symbols to be dropped: ['š', 'к', 'е', 'è', 'з', 'é', ';', 'ý', 'я', 'б', 'ğ', 'т', 'β', 'à', ')', 'с', 'о', ' ', 'č', 'у', '"', 'ю', 'н', '+', 'ø', "'", 'д', 'á', '³', 'ä', 'ъ', 'в', 'р', 'м', 'ř', 'а', '$', '%', 'ü', 'л', 'п', 'ш', 'и', ']', 'ě', 'ñ', 'í', 'ö']


In [9]:
symbols_to_replace = ["'", '"', ";", "%", "(", ")", "[", "]", "²", "€", "³", "·"]
for symb in symbols_to_replace:
    data_test_exp["FixedOutput"] = data_test_exp["FixedOutput"].apply(
        lambda x: x.replace(symb, "")
    )
    data_test_in["ASROutput"] = data_test_in["ASROutput"].apply(
        lambda x: x.replace(symb, "")
    )

data_test_exp["FixedOutput"] = data_test_exp["FixedOutput"].apply(rm_consecutive_spaces)
data_test_in["ASROutput"] = data_test_in["ASROutput"].apply(rm_consecutive_spaces)

In [10]:
with open("../../data/outputs/eda/train/symbols_to_replace.txt", "w", encoding="utf-8") as outfile:
    outfile.write("\n".join(symbols_to_replace))

In [11]:
symbols_to_drop_test=list(set(symbols_to_drop_test)-set(symbols_to_replace)-set(' '))
str(symbols_to_drop_test)

"['š', 'к', 'е', 'è', 'з', 'é', 'ý', 'я', 'б', 'ğ', 'т', 'β', 'à', 'о', 'с', 'č', 'у', 'ю', 'н', '+', 'ø', 'д', 'á', 'ä', 'ъ', 'в', 'р', 'м', 'ř', 'а', '$', 'ü', 'л', 'п', 'ш', 'и', 'ě', 'ñ', 'í', 'ö']"

In [12]:
with open("../../data/outputs/eda/test/non_polish_letters.txt", "w", encoding="utf-8") as outfile:
    outfile.write("\n".join(symbols_to_drop_test))

In [13]:
test_noise_rows = []
for i, row in data_test_exp.iterrows():
    if any(x in data_test_exp.loc[i, 'FixedOutput'] for x in symbols_to_drop_test):
        test_noise_rows.append(row['FileId'])

data_test_exp_noise = data_test_exp[data_test_exp.FileId.isin(test_noise_rows)]

# data_test_exp_noise.to_csv('../../data/outputs/eda/test/test_noise.csv')

data_test_exp_noise.head() # records to drop

,FileId,FixedOutput
12,wikinews179354,"prezydent usa george bush powiedział , że odno..."
26,wikinews184437,szef unii chrześcijańsko - demokratycznej i wi...
57,wikitalks005217,zgłoszenie zostało przeniesione z wikipedia : ...
60,wikinews185732,"świat obiegła informacja , że mehmet ali ağca ..."
61,wikinews218360,na terenach wystawowych w pradze w dniach 13 -...


In [14]:
len(data_test_exp_noise)

16

In [15]:
count_punctuation(data_test_exp_noise.set_index('FileId'), 'FixedOutput')

,fullstop,comma,question_mark,exclamation_mark,hyphen,colon,ellipsis
FileId,,,,,,,
wikinews179354,16.0,15.0,0.0,0.0,1.0,1.0,0.0
wikinews184437,10.0,11.0,0.0,0.0,1.0,0.0,0.0
wikitalks005217,23.0,20.0,4.0,0.0,1.0,15.0,0.0
wikinews185732,12.0,12.0,0.0,0.0,4.0,0.0,0.0
wikinews218360,25.0,10.0,0.0,0.0,4.0,0.0,0.0
wikitalks0014941,15.0,18.0,4.0,1.0,6.0,0.0,0.0
wikinews196753,7.0,16.0,0.0,0.0,0.0,3.0,0.0
wikinews183089,11.0,15.0,0.0,0.0,3.0,2.0,0.0
wikinews226381,11.0,15.0,0.0,0.0,0.0,3.0,0.0


In [16]:
records_with_noise_test = np.unique(np.array(data_test_exp_noise['FileId'].tolist()))
records_with_noise_test

array(['wikinews179354', 'wikinews180554', 'wikinews183089',
       'wikinews183185', 'wikinews184437', 'wikinews185273',
       'wikinews185732', 'wikinews196753', 'wikinews199816',
       'wikinews218360', 'wikinews226381', 'wikinews226751',
       'wikinews230002', 'wikinews231289', 'wikitalks0014941',
       'wikitalks005217'], dtype='<U16')

## <span style='font-family:Georgia'> Non-character symbols statistics summary

Count:
* how many symbols_to_drop contain each record
* how long is text in each record
* percent of symbols_to_drop in whole string in each record


In [17]:
# count number of symbols_to_drop in each record
fixed_output_test = data_test_exp_noise.loc[:, "FixedOutput"]
count_test = [0 for i in range(len(fixed_output_test))]
number_test = 0
for string in fixed_output_test:
    for letter in string:
        if letter in symbols_to_drop_test:
            count_test[number_test] = 1
    number_test += 1  # iterate for indexes, as from series only strings, without indexes

# count length of fixed output
length_test = [0 for i in range(len(fixed_output_test))]
number_test = 0
for string in fixed_output_test:
    length_test[number_test] = len(string)
    number_test += 1

df_fixed_output_test = pd.DataFrame(fixed_output_test)
df_fixed_output_test["length"] = length_test
df_fixed_output_test["count of symbols_to_drop"] = count_test
df_fixed_output_test["percent"] = [i / j * 100 for i, j in zip(count_test, length_test)]

df_fixed_output_test.head()

,FixedOutput,length,count of symbols_to_drop,percent
12,"prezydent usa george bush powiedział , że odno...",1790,1,0.055866
26,szef unii chrześcijańsko - demokratycznej i wi...,1129,1,0.088574
57,zgłoszenie zostało przeniesione z wikipedia : ...,1779,1,0.056211
60,"świat obiegła informacja , że mehmet ali ağca ...",1210,1,0.082645
61,na terenach wystawowych w pradze w dniach 13 -...,2299,1,0.043497


In [18]:
#  print whole records, which have at least 1% of characters to drop
to_print_test=df_fixed_output_test[df_fixed_output_test['percent']>1]
for i in range(len(to_print_test)):
    print(to_print_test.iloc[i,:].FixedOutput)
    print('\n')

### Words containing non-characters symbols preview

In [19]:
count_words_test=['' for i in range(len(fixed_output_test))]
number_test=0
for string in fixed_output_test:
    words_in_string_test=string.split(' ')
    for word in words_in_string_test:
        count_letters_test=0
        for letter in word:
            if letter in symbols_to_drop_test:
                count_letters_test+=1
        if count_letters_test>0:
            count_words_test[number_test]=count_words_test[number_test]+' '+ word
    number_test+=1
    
count_words_test = list(map(str.lstrip, count_words_test))
print(count_words_test)

['$', 'jiří čunek čunka jiří čunek jiří čunka', 'президент беларуси александр лукашенко указом объявил и июня днями траура', 'ağca ağca ağca ağcy ağcę', 'svět dwořáčkovej štípkova svět svět', 'βy βy βy', '+', 'cuauhtémoc', 'gällivare bjørgen', 'alizée alizée mylène alizée à alizée', 'bjørndalena', 'björna jürgenowi björn sébastien söderling martín argüello karanušić martín cañas hernández josé jürgen jiří vaněk stéphane', 'mikuláša gašparoviča vladimír lipšic hrušovský', 'léon', 'düsseldorfie düsseldorfie düsseldorfu', 'tägtgren']


In [20]:
with open("../../data/outputs/eda/test/noisy_words.txt", "w", encoding="utf-8") as outfile:
    outfile.write("\n".join(count_words_test))

In [21]:
pd.options.display.max_colwidth = None

df_fixed_output_test['words containing']=count_words_test
df_fixed_output_test

,FixedOutput,length,count of symbols_to_drop,percent,words containing
12,"prezydent usa george bush powiedział , że odnowa nowego orleanu i innych zalanych terenów potrwa lata . stan luizjana , wraz ze stolicą nowym orleanem , ucierpiał w wyniku ataku huraganu katrina . prezydent skrócił swoje wakacje w teksasie oraz zwołał posiedzenie gabinetu , który obejmie kontrolę nad nadzorem akcji ratunkowej jednej z najgorszych katastrof naturalnych . bush wyznaczył trzy priorytety dotyczące walki ze skutkami kataklizmu : ratowanie życia ofiarom , w tym poszukiwanie zaginionych pomoc poszkodowanym i odnalezionym odbudowa i usuwanie szkód w ramach akcji ratunkowej przydzielono dodatkowo 10 tysięcy żołnierzy , którzy wspomogą operację w regionach najbardziej dotkniętych katastrofą . zaliczają się do nich części stanów luizjany i mississippi . służby medyczne zaoferowały ponad 10 tysięcy łóżek , zaś armia amerykańska wysyła swoje helikoptery i łodzie aby uratować mieszkańców znajdujących się w schronieniu . późnym wieczorem , w środę , rozpoczęła się ewakuacja reszty mieszkańców nowego orleanu - tych , którzy pozostali w schronach oraz na stadionie superdome w centrum miasta . ich liczbę szacuje się na 20000 ludzie są wywożeni do oddalonego o 350 mil stadionu astrodome w houston . w tym celu podstawiono 300 autobusów . niektórzy wykorzystują sytuację i w poszukiwaniu jedzenia lub cennych rzeczy włamują się do domów , biur , szpitali . rozbijają szyby , wyłamują drzwi . wleką za sobą to , co udaje się im unieść bądź przenieść . uderzenie huraganu katrina spowodowało wzrost cen benzyny w wielu częściach stanów zjednoczonych . osiągnęła ona wartość powyżej 3 dolarów za galon . w niektórych stanach wynosi ona 3 50 $ . nad zatoką meksykańską umiejscowiony jest jeden z największych okręgów przemysłowych , mieszczącym wiele rafinerii oraz rurociągów .",1790,1,0.055866,$
26,"szef unii chrześcijańsko - demokratycznej i wicepremier czech , jiří čunek poinformował w czwartek , że 7 listopada poda się do dymisji . jego decyzja jest spowodowana zarzutami jakie wobec niego wysunęła czeska prokuratura . zarzuca się mu , że w latach 90 jego rodzina nielegalnie pobierała świadczenia socjalne . w tym samym okresie na koncie čunka miało znajdować się ponad 3 , 5 mln koron . dodatkowo jiří čunek zataił przed urzędem podatkowym to , iż posiada tak znaczne dochody . już wcześniej czeskiemu wicepremierowi stawiano aferalne zarzuty . oskarżano go o to , że będąc starosta morawskiej vesztiny przyjął półmilionową łapówkę . miało to być wynagrodzenia ze strony pewnej firmy , której umożliwiono wykup pakietu większościowego w komunalnym przedsiębiorstwie zajmującym się budową mieszkań . udało mu się te zarzuty oddalić , jednak wpisał się nimi do historii czech jako pierwszy członek rządu , którego oskarżono o korupcje . w obliczu obecnej afery jiří čunka złożył dymisję ze stanowiska w rządzie , jednocześnie zapowiadając , że nie ma zamiaru rezygnować z przewodniczenie swojej partii i mandatu senatora .",1129,1,0.088574,jiří čunek čunka jiří čunek jiří čunka
57,"zgłoszenie zostało przeniesione z wikipedia : zgłoś błąd w artykule ponieważ prawdopodobnie nie zostało rozwiązane w ciągu 45 dni . 1 po co w tabelce ofiar rubryka fotografia , która jest kompletnie pusta ? 2 tu jest mowa o dwudniowej żałobie , nie trzydniowej . zgłasza : żyrafał 11 : 51 , 25 paź 2014 co to za miejsce pracy nauki komórka ? usunąłem zbędną kolumnę fotografia . natomiast nie wiem , co z tą żałobą . na en . wiki też jest trzydniowa . koveraslupus 14 : 31 , 25 paź 2014 na ru . wiki mamy dwa : президент беларуси александр лукашенко указом объявил 1 и 2 июня днями траура . a na en . wiki szablon o niepełnym uźródłowieniu . żyrafał 15 : 29 , 25 paź 2014 to co pisze enwiki i ruwiki kompletnie nas nie powinno interesować , bo wikipedia nie może być źródłem sama dla siebie . ma być podane to co dają zewnętrzne wiarygodne źródła , a jak takich nie ma to fragment usunąć . aotearoa dyskusja 15 : 51 , 25 p